In [ ]:
import numpy_financial as npf

pv = 300000
r = .04
term = 360

for month in range(1, term+1):

    pmt = f"${npf.pmt(r/12, term, -pv):,.2f}"
    ppmt = f"${npf.ppmt(r/12, month, term, -pv):,.2f}"
    ipmt = f"${npf.ipmt(r/12, month, term, -pv):,.2f}"




    rint = (f"{pmt:15}{ipmt:15}{ppmt:15}")
    print(rint)
    


In [20]:
import datetime as dt
from dateutil.relativedelta import relativedelta
import matplotlib.pyplot as plt
import numpy_financial as npf
import pandas as pd

class Loan:
    
    def __init__(self, rate, term, loan_amount, start=dt.date.today().isoformat()):
        self.rate = rate / 1200
        self.period = term * 12
        self.loan_amount = loan_amount
        self.start = dt.date.fromisoformat('2022-07-01') + relativedelta(months=1)
        self.pmt = npf.pmt(self.rate, self.period, -self.loan_amount)
        self.pmt_str = f"${self.pmt:,.2f}"
        self.table = self.loan_table()

    def loan_table(self):
        period = [self.start + relativedelta(months=x) for x in range(self.period)]
        interest = [npf.ipmt(self.rate, month, self.period, -self.loan_amount)
                    for month in range(1, self.period + 1)]
        principal = [npf.ppmt(self.rate, month, self.period, -self.loan_amount)
                     for month in range(1, self.period + 1)]
        table = pd.DataFrame({'Payment': self.pmt,
                              'Interest': interest,
                              'Principal': principal}, index=pd.to_datetime(period))
        table['Balance'] = self.loan_amount - table['Principal'].cumsum()
        return table.round(2)


loan = Loan(5, 100, 3600000, dt.date.today().isoformat())
print(loan.table)

             Payment            Interest  Principal     Balance
2022-08-01  15102.82             15000.0     102.82  3599897.18
2022-09-01  15102.82  14999.571564053425     103.25  3599793.92
2022-10-01  15102.82  14999.141342957075     103.68  3599690.24
2022-11-01  15102.82  14998.709329272824     104.12  3599586.12
2022-12-01  15102.82  14998.275515531552     104.55  3599481.57
...              ...                 ...        ...         ...
2122-03-01  15102.82  310.74707397669556   14792.08    59787.22
2122-04-01  15102.82  249.11341750522453   14853.71    44933.51
2122-05-01  15102.82   187.2229541314145   14915.60    30017.91
2122-06-01  15102.82  125.07461382697026   14977.75    15040.16
2122-07-01  15102.82   62.66732210467259   15040.16        0.00

[1200 rows x 4 columns]


In [21]:
loan.table.to_csv('loan_table.csv', index=True)
df = pd.read_csv('loan_table.csv')
df.rename(columns={'Unnamed: 0': 'Date'}, inplace=True)

In [22]:
df

,Date,Payment,Interest,Principal,Balance
0,2022-08-01,15102.82,15000.000000,102.82,3599897.18
1,2022-09-01,15102.82,14999.571564,103.25,3599793.92
2,2022-10-01,15102.82,14999.141343,103.68,3599690.24
3,2022-11-01,15102.82,14998.709329,104.12,3599586.12
4,2022-12-01,15102.82,14998.275516,104.55,3599481.57
...,...,...,...,...,...
1195,2122-03-01,15102.82,310.747074,14792.08,59787.22
1196,2122-04-01,15102.82,249.113418,14853.71,44933.51
1197,2122-05-01,15102.82,187.222954,14915.60,30017.91
1198,2122-06-01,15102.82,125.074614,14977.75,15040.16


In [23]:
df.describe()

,Payment,Interest,Principal,Balance
count,1.200000e+03,1200.000000,1200.000000,1.200000e+03
mean,1.510282e+04,12102.824627,3000.000125,2.901678e+06
std,3.639496e-12,3711.136231,3711.136204,8.943838e+05
min,1.510282e+04,62.667322,102.820000,0.000000e+00
25%,1.510282e+04,10778.042095,357.590000,2.582405e+06
50%,1.510282e+04,13859.238984,1243.585000,3.324974e+06
75%,1.510282e+04,14745.233565,4324.780000,3.538498e+06
max,1.510282e+04,15000.000000,15040.160000,3.599897e+06
